# Data Process

In [2]:
import numpy as np
import pandas as pd
import csv

### Reading data

In [99]:
data_file_path          = './train_data_first/'
data_file_name          = 'public_train_x_{fname}_full_hashed.csv'
information_file_name   = 'custinfo'
credit_file_name        = 'ccba'
consumption_file_name   = 'cdtx0001'
remit_file_name         = 'remit1'
trading_file_name       = 'dp'

In [130]:
information_data    = pd.read_csv(data_file_path + data_file_name.format(fname = information_file_name))
credit_data         = pd.read_csv(data_file_path + data_file_name.format(fname = credit_file_name), index_col='cust_id')
consumption_data    = pd.read_csv(data_file_path + data_file_name.format(fname = consumption_file_name), index_col = 'cust_id')
remit_data          = pd.read_csv(data_file_path + data_file_name.format(fname = remit_file_name), index_col = 'cust_id')
trading_data        = pd.read_csv(data_file_path + data_file_name.format(fname = trading_file_name), index_col = 'cust_id')
train_x = pd.read_csv(data_file_path + 'train_x_alert_date.csv')
train_y = pd.read_csv(data_file_path + 'train_y_answer.csv')
public_x = pd.read_csv(data_file_path + 'public_x_alert_date.csv')
public_y = pd.read_csv('./24_ESun_public_y_answer.csv')

### Load training data x

In [101]:
train_x_alert_key = []
train_x_alert_date = []
train_x_cust_id = []
train_x_size = len(train_x)
for i in range(train_x_size):
    train_x_alert_key.append(train_x['alert_key'][i])
    train_x_alert_date.append(train_x['date'][i])
    train_x_cust_id.append(information_data['cust_id'][i + 1845])
train_x_alert_date = [x for _, x in sorted(zip(train_x_alert_key, train_x_alert_date))]
train_x_alert_key.sort()


### Load training data y

In [119]:
train_y_alert_key = []
train_y_sar_flag = []
train_y_size = len(train_y)
for i in range(train_y_size):
    train_y_alert_key.append(train_y['alert_key'][i])
    train_y_sar_flag.append(train_y['sar_flag'][i])
train_y_sar_flag = [y for _, y in sorted(zip(train_y_alert_key, train_y_sar_flag))]
train_y_alert_key.sort()

### $V_{info}$

In [89]:
V_info = []
for i in range(train_x_size):
    risk_rank       = information_data['risk_rank'][i + 1845]
    occupation_code = information_data['occupation_code'][i + 1845]
    total_asset     = information_data['total_asset'][i + 1845]
    AGE             = information_data['AGE'][i + 1845]
    V_info.append([risk_rank, occupation_code, total_asset, AGE])

### $V_{cred}$

In [94]:
V_cred = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(train_x_size):
    V_cred_i = []
    if train_x_cust_id[i] not in credit_data.index:
        V_cred_i = [0] * 117
    # search_cust = credit_data[credit_data['cust_id'] == train_x_cust_id[i]]
    else:
        search_cust = credit_data.loc[[train_x_cust_id[i]]]
        for m in month:
            search_month = search_cust[search_cust['byymm'] == m]
            if len(search_month.to_numpy()) == 0:
                V_cred_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            elif m > train_x_alert_date[i]:
                V_cred_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            else:
                lupay = search_month['lupay'].to_numpy()[0]
                cycam = search_month['cycam'].to_numpy()[0]
                usgam = search_month['usgam'].to_numpy()[0]
                clamt = search_month['clamt'].to_numpy()[0]
                csamt = search_month['csamt'].to_numpy()[0]
                inamt = search_month['inamt'].to_numpy()[0]
                cucsm = search_month['cucsm'].to_numpy()[0]
                cucah = search_month['cucah'].to_numpy()[0]
                V_cred_i.extend([m, lupay, cycam, usgam, clamt, csamt, inamt, cucsm, cucah])
            
    V_cred.append(V_cred_i)

### $V_{cons}$

In [103]:
V_cons = []
day = range(1, 394)
for i in range(train_x_size):
    V_cons_i = []
    if train_x_cust_id[i] not in consumption_data.index:
        V_cons_i = [0] * 5 * 393
    else:
        search_cust = consumption_data.loc[[train_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['date'] == d]
            n = len(search_day.to_numpy())
            if d > train_x_alert_date[i]:
                V_cons_i.extend([0, 0, 0, 0, 0])
            elif n == 0:
                V_cons_i.extend([d, 0, 0, 0, 0])
            else:
                country = sum(search_day['country'].to_numpy())
                cur_type = sum(search_day['cur_type'].to_numpy())
                amt = sum(search_day['amt'].to_numpy())
                V_cons_i.extend([d, n, country, cur_type, amt])
            
    V_cons.append(V_cons_i)

### $V_{remit}$

In [112]:
V_remit = []
day = range(1, 394)
for i in range(train_x_size):
    V_remit_i = []
    if train_x_cust_id[i] not in remit_data.index:
        V_remit_i = [0] * 4 * 393
    else:
        search_cust = remit_data.loc[[train_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['trans_date'] == d]
            n = len(search_day.to_numpy())
            if d > train_x_alert_date[i]:
                V_remit_i.extend([0, 0, 0, 0])
            elif n == 0:
                V_remit_i.extend([d, 0, 0, 0])
            else:
                trans_no = sum(search_day['trans_no'].to_numpy())
                trade_amount_usd = sum(search_day['trade_amount_usd'].to_numpy())
                V_remit_i.extend([d, n, trans_no, trade_amount_usd])
        
    V_remit.append(V_remit_i)

### $V_{trade}$

In [115]:
V_trade = []
day = range(1, 394)
for i in range(train_x_size):
    V_trade_i = []
    if train_x_cust_id[i] not in trading_data.index:
        V_trade_i = [0] * 9 * 393
    else:
        search_cust = trading_data.loc[[train_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['tx_date'] == d]
            n = len(search_day.to_numpy())
            if d > train_x_alert_date[i]:
                V_trade_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            elif n == 0:
                V_trade_i.extend([d, 0, 0, 0, 0, 0, 0, 0, 0])
            else:
                # debit_credit = sum(search_day['debit_credit'].to_numpy())
                tx_type = sum(search_day['tx_type'].to_numpy())
                tx_amt = sum(search_day['tx_amt'].to_numpy())
                info_asset_code = sum(search_day['info_asset_code'].to_numpy())
                fiscTxId = sum(search_day['fiscTxId'].to_numpy())
                txbranch = sum(search_day['txbranch'].to_numpy())
                cross_bank = sum(search_day['cross_bank'].to_numpy())
                ATM = sum(search_day['ATM'].to_numpy())
                V_trade_i.extend([d, n, tx_type, tx_amt, info_asset_code, fiscTxId, txbranch, cross_bank, ATM])
        
    V_trade.append(V_trade_i)

### Store processed data

In [116]:
pd.DataFrame(V_info).to_csv('./train_data_processed/V_info.csv')
pd.DataFrame(V_cred).to_csv('./train_data_processed/V_cred.csv')
pd.DataFrame(V_cons).to_csv('./train_data_processed/V_cons.csv')
pd.DataFrame(V_remit).to_csv('./train_data_processed/V_remit.csv')
pd.DataFrame(V_trade).to_csv('./train_data_processed/V_trade.csv')

In [120]:
pd.DataFrame(train_y_sar_flag).to_csv('./train_data_processed/train_y.csv')

### Public data

In [133]:
public_x_alert_key = []
public_x_alert_date = []
public_x_cust_id = []
public_x_size = len(public_x)
for i in range(public_x_size):
    public_x_alert_key.append(public_x['alert_key'][i])
    public_x_alert_date.append(public_x['date'][i])
    public_x_cust_id.append(information_data['cust_id'][i])
public_x_alert_date = [x for _, x in sorted(zip(public_x_alert_key, public_x_alert_date))]
public_x_alert_key.sort()

In [134]:
public_y_alert_key = []
public_y_sar_flag = []
public_y_size = len(public_y)
for i in range(public_y_size):
    public_y_alert_key.append(public_y['alert_key'][i])
    public_y_sar_flag.append(public_y['sar_flag'][i])
public_y_sar_flag = [y for _, y in sorted(zip(public_y_alert_key, public_y_sar_flag))]
public_y_alert_key.sort()

In [135]:
V_info_public = []
for i in range(public_x_size):
    risk_rank       = information_data['risk_rank'][i]
    occupation_code = information_data['occupation_code'][i]
    total_asset     = information_data['total_asset'][i]
    AGE             = information_data['AGE'][i]
    V_info_public.append([risk_rank, occupation_code, total_asset, AGE])

In [136]:
V_cred_public = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(public_x_size):
    V_cred_public_i = []
    if public_x_cust_id[i] not in credit_data.index:
        V_cred_public_i = [0] * 117
    # search_cust = credit_data[credit_data['cust_id'] == train_x_cust_id[i]]
    else:
        search_cust = credit_data.loc[[public_x_cust_id[i]]]
        for m in month:
            search_month = search_cust[search_cust['byymm'] == m]
            if len(search_month.to_numpy()) == 0:
                V_cred_public_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            elif m > public_x_alert_date[i]:
                V_cred_public_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            else:
                lupay = search_month['lupay'].to_numpy()[0]
                cycam = search_month['cycam'].to_numpy()[0]
                usgam = search_month['usgam'].to_numpy()[0]
                clamt = search_month['clamt'].to_numpy()[0]
                csamt = search_month['csamt'].to_numpy()[0]
                inamt = search_month['inamt'].to_numpy()[0]
                cucsm = search_month['cucsm'].to_numpy()[0]
                cucah = search_month['cucah'].to_numpy()[0]
                V_cred_public_i.extend([m, lupay, cycam, usgam, clamt, csamt, inamt, cucsm, cucah])
            
    V_cred_public.append(V_cred_public_i)

In [137]:
V_cons_public = []
day = range(1, 394)
for i in range(public_x_size):
    V_cons_public_i = []
    if public_x_cust_id[i] not in consumption_data.index:
        V_cons_public_i = [0] * 5 * 393
    else:
        search_cust = consumption_data.loc[[public_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['date'] == d]
            n = len(search_day.to_numpy())
            if d > public_x_alert_date[i]:
                V_cons_public_i.extend([0, 0, 0, 0, 0])
            elif n == 0:
                V_cons_public_i.extend([d, 0, 0, 0, 0])
            else:
                country = sum(search_day['country'].to_numpy())
                cur_type = sum(search_day['cur_type'].to_numpy())
                amt = sum(search_day['amt'].to_numpy())
                V_cons_public_i.extend([d, n, country, cur_type, amt])
            
    V_cons_public.append(V_cons_public_i)

In [138]:
V_remit_public = []
day = range(1, 394)
for i in range(public_x_size):
    V_remit_public_i = []
    if public_x_cust_id[i] not in remit_data.index:
        V_remit_public_i = [0] * 4 * 393
    else:
        search_cust = remit_data.loc[[public_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['trans_date'] == d]
            n = len(search_day.to_numpy())
            if d > public_x_alert_date[i]:
                V_remit_public_i.extend([0, 0, 0, 0])
            elif n == 0:
                V_remit_public_i.extend([d, 0, 0, 0])
            else:
                trans_no = sum(search_day['trans_no'].to_numpy())
                trade_amount_usd = sum(search_day['trade_amount_usd'].to_numpy())
                V_remit_public_i.extend([d, n, trans_no, trade_amount_usd])
        
    V_remit_public.append(V_remit_public_i)

In [139]:
V_trade_public = []
day = range(1, 394)
for i in range(public_x_size):
    V_trade_public_i = []
    if public_x_cust_id[i] not in trading_data.index:
        V_trade_public_i = [0] * 9 * 393
    else:
        search_cust = trading_data.loc[[public_x_cust_id[i]]]
        for d in day:
            search_day = search_cust[search_cust['tx_date'] == d]
            n = len(search_day.to_numpy())
            if d > public_x_alert_date[i]:
                V_trade_public_i.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
            elif n == 0:
                V_trade_public_i.extend([d, 0, 0, 0, 0, 0, 0, 0, 0])
            else:
                # debit_credit = sum(search_day['debit_credit'].to_numpy())
                tx_type = sum(search_day['tx_type'].to_numpy())
                tx_amt = sum(search_day['tx_amt'].to_numpy())
                info_asset_code = sum(search_day['info_asset_code'].to_numpy())
                fiscTxId = sum(search_day['fiscTxId'].to_numpy())
                txbranch = sum(search_day['txbranch'].to_numpy())
                cross_bank = sum(search_day['cross_bank'].to_numpy())
                ATM = sum(search_day['ATM'].to_numpy())
                V_trade_public_i.extend([d, n, tx_type, tx_amt, info_asset_code, fiscTxId, txbranch, cross_bank, ATM])
        
    V_trade_public.append(V_trade_public_i)

In [140]:
pd.DataFrame(V_info_public).to_csv('./public_data_processed/V_info_public.csv')
pd.DataFrame(V_cred_public).to_csv('./public_data_processed/V_cred_public.csv')
pd.DataFrame(V_cons_public).to_csv('./public_data_processed/V_cons_public.csv')
pd.DataFrame(V_remit_public).to_csv('./public_data_processed/V_remit_public.csv')
pd.DataFrame(V_trade_public).to_csv('./public_data_processed/V_trade_public.csv')

In [141]:
pd.DataFrame(public_y_sar_flag).to_csv('./public_data_processed/public_y.csv')

In [85]:
search_cust1 = credit_data.loc['55e6688f0acbaa20739fe919ba0ac9d7a00430272779c36c9bdde188aea37682']
search_cust2 = credit_data.loc['3f97df758617152c9ebda71bfb4922c038a3a274db5faa0b570a07f02672aedc']

month = [0]
print(type(search_cust1))
print(type(search_cust2))
# for m in month:
#     search_month = search_cust[search_cust['byymm'] == 365]

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
